In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import SAM
import numpy as np

model = SAM('sam_b.pt')

image_path = './data/test_img_R.jpg' 
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
results = model(image_rgb)






In [ ]:
if results and results[0].masks is not None:
    # Get the raw masks as a numpy array (HxWxN, where N is number of masks)
    masks_tensor = results[0].masks.data
    masks_np = masks_tensor.cpu().numpy() # Move to CPU and convert to NumPy

    # Create a blank image to draw all masks on
    masked_image = np.zeros_like(image_rgb, dtype=np.uint8)

    # You can iterate through masks and draw them with random colors
    for i, mask in enumerate(masks_np):
        color = np.random.randint(0, 255, 3, dtype=np.uint8)
        # Apply the mask: wherever mask is True, apply the color
        masked_image[mask] = color

    # You can blend the original image with the masks for a better visualization
    alpha = 0.5 # Transparency of the masks
    blended_image = cv2.addWeighted(image_rgb, 1, masked_image, alpha, 0)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(blended_image)
    plt.title("Segmented Image")
    plt.axis('off')
    plt.show()

    print(f"Found {len(masks_np)} segments.")

else:
    print("No segments found or an issue occurred during segmentation.")